In [17]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
%matplotlib inline

## Using a finite volume method to rewrite Equation 122 of Ogilvie 1999 in terms of a conservative equation

### $\vec{\lambda} \equiv I\Omega r^2\vec{l}$
### $I \equiv \frac{1}{2\pi}\int_0^{2\pi}\int_{-\infty}^{\infty}\rho z^2 dz d\phi = \Sigma (h/r)^2r^2$ (if $\gamma = 1$) 
### $\Omega = r^{-3/2}$ (Keplerian, $G=M=1$)

### Internal coordinates: $x = {\rm ln}\,r$, $\frac{\partial}{\partial r} = \frac{1}{r}\frac{\partial}{\partial x}$

### Warp parameter: $\psi \equiv |\frac{\partial}{\partial x}\frac{\vec{\lambda}}{\lambda}|$
### Dispersion coefficients: $Q_1 = Q_1(\psi),\,Q_2 = Q_2(\psi),\,Q_3 = Q_3(\psi)$ (interpolated from table)

### $\frac{\partial \vec{\lambda}}{\partial t} + \frac{\partial}{\partial x}\left[F_{\rm adv}(\vec{\lambda}) + F_{\rm diff}(\vec{\lambda})\right] = \vec{S}$

### $\vec{S} = \vec{T}(h/r)^2r^2$ is some source term, where $\vec{T}$ is the same quantity in Equation 122 of Ogilvie 1999 and for the case of Lense-Thirring precession:
### $\vec{S} = \vec{\Omega}_p\times\vec{\lambda}$ where $\vec{\Omega}_p = 2a/r^3\vec{z}$ where $a$ is the dimensionless spin ($-1 < a < 1$) of the central black hole, where we have defined the black hole angular momentum to be in the $\vec{z}$ direction

### $F_{\rm adv}$ is the advective flux term:
### $F_{\rm adv} = -r^{-3/2}(Q_1(\psi) + 2Q_2(\psi)\psi^2)\vec{\lambda}$

### $F_{\rm diff}$ is the diffusive flux term:
### $F_{\rm diff} = \frac{2\vec{\lambda}}{\lambda}\frac{\partial}{\partial x}(Q_1(\psi)\lambda r^{-3/2}) - Q_2(\psi)r^{-3/2}\lambda\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda}) - Q_3(\psi)r^{-3/2}\vec{\lambda}\times\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})$

### Where for convenience we write the components of the cross product explicitly:

### $(\vec{\lambda}\times\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda}))_x = \lambda_y\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_z - \lambda_z\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_y$

### $(\vec{\lambda}\times\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda}))_y = \lambda_z\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_x - \lambda_x\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_z$

### $(\vec{\lambda}\times\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda}))_z = \lambda_x\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_y - \lambda_y\frac{\partial}{\partial x}(\frac{\vec{\lambda}}{\lambda})_x$

In [16]:
test =  np.logspace(np.log(1),np.log(10),10,base=np.exp(1))
print test
print np.average(np.log(test)[1:] - np.log(test)[:-1]), np.log(test)[1:] - np.log(test)[:-1]

test2 =  np.logspace(np.log10(1),np.log10(10),10)
print  np.exp(test)
print np.log10(test2)
print 2**(2.)*2

[ 1.          1.29154967  1.66810054  2.15443469  2.7825594   3.59381366
  4.64158883  5.9948425   7.74263683 10.        ]
0.2558427881104495 [0.25584279 0.25584279 0.25584279 0.25584279 0.25584279 0.25584279
 0.25584279 0.25584279 0.25584279]
[2.71828183e+00 3.63842052e+00 5.30208711e+00 8.62301413e+00
 1.61603288e+01 3.63725243e+01 1.03708993e+02 4.01353467e+02
 2.30454105e+03 2.20264658e+04]
[0.         0.11111111 0.22222222 0.33333333 0.44444444 0.55555556
 0.66666667 0.77777778 0.88888889 1.        ]
8.0
